##Step 1: Install Required Packages

In [59]:
!pip install flask flask-socketio pyngrok pdfminer.six

##Step 2: Configure ngrok with Your Authtoken

In [63]:
!ngrok config add-authtoken 2ohlscwY8i3ScWTdWjwffURXGoC_5dkVKAXMieiaZHN16Ax6Z

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [64]:
!pkill ngrok

INFO:werkzeug:127.0.0.1 - - [11/Nov/2024 13:05:21] "GET /socket.io/?EIO=4&transport=websocket&sid=6RmTUH6tbKrIfO37AAAA HTTP/1.1" 200 -


##Step 3: Create the Flask Server

In [69]:
import os
from flask import Flask, render_template
from flask_socketio import SocketIO, emit
from pyngrok import ngrok

# Initialize Flask app and SocketIO
app = Flask(__name__)
socketio = SocketIO(app)

# Global variable to track the current PDF page
current_page = 1

# Expose the app via ngrok
public_url = ngrok.connect(5001)
print(f"Public URL: {public_url}")

# Route to serve the HTML page
@app.route('/')
def index():
    return render_template('index.html')

# Admin changing page event
@socketio.on('admin-change-page')
def handle_admin_change_page(page):
    global current_page
    current_page = page
    emit('page-change', page, broadcast=True)

# Viewer changing page event
@socketio.on('viewer-change-page')
def handle_viewer_change_page(page):
    emit('page-change', page, broadcast=True)

# Start the server
def run_server():
    socketio.run(app, port=5000)


Public URL: NgrokTunnel: "https://50c4-104-196-226-1.ngrok-free.app" -> "http://localhost:5001"


##Step 4: Create HTML Template for Frontend

In [70]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>PDF Co-Viewer</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.7.2/socket.io.js"></script>
    <script src="https://mozilla.github.io/pdf.js/build/pdf.js"></script>
    <style>
        #pdf-render {
            border: 1px solid black;
            width: 80%;
            height: 500px;
            margin: 20px auto;
            overflow: auto;
        }
        .controls {
            text-align: center;
        }
        button {
            padding: 10px;
            margin: 10px;
        }
    </style>
</head>
<body>
    <h1 style="text-align:center;">PDF Co-Viewer</h1>
    <div id="pdf-render"></div>
    <div class="controls">
        <button onclick="changePage(-1)">Previous</button>
        <span id="page-info">Page 1</span>
        <button onclick="changePage(1)">Next</button>
    </div>

    <script>
        const socket = io.connect(window.location.origin);
        let currentPage = 1;
        const url = 'https://arxiv.org/pdf/1706.03762.pdf'; // Example PDF URL

        const pdfjsLib = window['pdfjs-dist/build/pdf'];
        pdfjsLib.GlobalWorkerOptions.workerSrc = '//mozilla.github.io/pdf.js/build/pdf.worker.js';

        let pdfDoc = null;

        // Load PDF
        pdfjsLib.getDocument(url).promise.then(doc => {
            pdfDoc = doc;
            renderPage(currentPage);
        });

        // Render PDF Page
        function renderPage(pageNum) {
            pdfDoc.getPage(pageNum).then(page => {
                const viewport = page.getViewport({ scale: 1.5 });
                const canvas = document.createElement('canvas');
                const ctx = canvas.getContext('2d');
                canvas.height = viewport.height;
                canvas.width = viewport.width;

                const pdfContainer = document.getElementById('pdf-render');
                pdfContainer.innerHTML = '';
                pdfContainer.appendChild(canvas);

                const renderContext = {
                    canvasContext: ctx,
                    viewport: viewport
                };
                page.render(renderContext);
                document.getElementById('page-info').textContent = `Page ${pageNum} of ${pdfDoc.numPages}`;
            });
        }

        // Change Page
        function changePage(offset) {
            const newPage = currentPage + offset;
            if (newPage > 0 && newPage <= pdfDoc.numPages) {
                currentPage = newPage;
                renderPage(currentPage);
                socket.emit('admin-change-page', currentPage);
            }
        }

        // Listen for page changes
        socket.on('page-change', (page) => {
            currentPage = page;
            renderPage(page);
        });
    </script>
</body>
</html>


Overwriting templates/index.html


##Step 5: Start the Flask Server

In [71]:
# Updated run_server function
def run_server():
    socketio.run(app, port=5001, allow_unsafe_werkzeug=True)

# Start the Flask server with SocketIO in a non-blocking way
import threading

server_thread = threading.Thread(target=run_server)
server_thread.start()



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
